In [1]:
#!/usr/bin/env python

In [2]:
__author__ = "Tariq Faquih"
__copyright__ = "Copyright 2020, Clinical Epidemiology Department, LUMC"
__credits__ = ["Tariq Faquih", "Linda Nab", "Ype Jong"]
__license__ = "MIT License"
__maintainer__ = "Tariq Faquiih"
__email__ = "t.o.faquih@lumc.nl"
__status__ = "Development"

# Import modules

In [3]:
import json, csv , os , sys , datetime
from Bio import Entrez
from Bio import Medline



# COVID class

This class objects send pubmed queries through the API and stores the output in json files

In [7]:

class COVID:
 
    def __init__(self , startD , endD):

        #list of query terms to be used in the search_function
        querylist = ('' , 
                 '(NEJM[journal] OR BMJ[journal] OR lancet[journal] OR nature[journal] OR JAMA[journal])', 
                 'elderly[TITLE]',
                 'clinical trial[Title/Abstract]' , 
                 'italy[Title/Abstract]' , 
                 'netherlands[Title/Abstract]' , 
                 'case control study' , 
                 'epidemiology' , 
                 'mortality', 
                 'pregnant[TITLE]',
                 '(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])' )
        
        #json_file stores the proper json format to be used in the googlesheet
        #dict_file stores the output in a dictionary to be loaded in later uses
        json_file ='results2.json'
        dict_file ='results_dictionary.json'

        #read the stored dictionary file (dict_file) or create a new blank dictionary
        if os.path.isfile(dict_file):
            self.mainDict = json.load(open(dict_file))
        else:
            self.mainDict = {}
        
        #set counter for how many articles are added and create a list to store the log messages
        self.NumNew = 0
        self.Log = []
        
        #for each query term in the querylist, run the search function using the provideed start
        #and end dates
        for X in querylist:
            self.search_function(X , startD , endD )
        
        #add the total number of added articles to the log list
        self.Log.append(self.NumNew)
        
        #Write the main json file
        with open(json_file , 'w') as fp:   
            Output=[]
            for Key,Item in self.mainDict.items():
                Output.append(Item)
            json.dump(Output, fp)
            
        #Write the exact dict as json file (easily read by the script)
        with open(dict_file , 'w') as fp:   
            json.dump(self.mainDict, fp)
        
        with open('log_{}_{}.txt'.format(startD.replace('/' , '') , endD.replace('/' , '')) ,'w'  , newline='') as fp:
            W = csv.writer(fp)
            W.writerow(['Search results for range {} to {}'.format(startD , endD)])
            W.writerow(['Number of Records Added: {}'.format(self.Log[-1])])
            for Line in self.Log[:-1]:
                W.writerow([Line])
        

    def search_function (self , MyTerms, startD , endD):

        Entrez.email = "tariqf549@gmail.com"
        MainTerm = """(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] ))))"""

        #MainTerm = '"COVID-19"[All Fields]'
        DateRange = '"{}"[MHDA] : "{}"[MHDA]'.format(startD , endD)
        Query = MainTerm + ' AND ' + MyTerms + ' AND ' + DateRange
        print(Query)
        self.Log.append(Query)
        search_results = Entrez.read(
            Entrez.esearch(
                db="pubmed", term=Query,  datetype="pdat", usehistory="y" , sort = 'relevance' 
            )
        )
        count = int(search_results["Count"])
        self.Log.append("Found %i results" % count)
        
        print("Found %i results" % count)

        batch_size = 10
        out_handle = open("corona_{}_papers.txt".format(MyTerms.split('[')[0]), "w")
        for start in range(0, count, batch_size):
            end = min(count, start + batch_size)
            print("Going to download record %i to %i" % (start + 1, end))
            self.Log.append("Going to download record %i to %i" % (start + 1, end))
            fetch_handle = Entrez.efetch(
                db="pubmed",
                rettype="medline",
                retmode="text",
                retstart=start,
                retmax=batch_size,
                webenv=search_results["WebEnv"],
                query_key=search_results["QueryKey"],
            )
            data = fetch_handle.read()

            dataresults = data.split('\nPMID')[1:]
            self.add2dict(dataresults , MyTerms)

            fetch_handle.close()
            out_handle.write(data)
        out_handle.close()
        
        
    def add2dict(self , dataresults , Q):
        for hit in dataresults:
            m1 = 'PMID' + hit
            parse_res = Medline.read(m1.split('\n'))
            PMID = parse_res['PMID']
            Tag = Q
            if PMID in self.mainDict.keys():
                print('PMID [{}] exists'.format(PMID))
                self.Log.append('PMID exists')
                if Q not in self.mainDict[PMID]['Tag']:
                     self.mainDict[PMID]['Tag'] = self.mainDict[PMID]['Tag']+';'+Tag
                continue
            else:
                Title = parse_res['TI']
                dateP = parse_res['DP']
                dateC = parse_res['MHDA']

                if 'JT' in parse_res.keys():
                    JournalName  = parse_res['JT']
                else: JournalName = ''

                #if 'LR' in parse_res.keys():
                #    dateMod  = parse_res['LR']
                #else: dateMod = ''

                if 'AB' in parse_res.keys():
                    Abstract = parse_res['AB']
                    tempAbs = Abstract.split(' ')
                    lastN=0
                    newabs = []
                    for N in range(30, len(tempAbs)+30, 30) :
                        newabs.append(' '.join(tempAbs[lastN:N]))
                        lastN= N
                    Abstract = '\n'.join(newabs)


                else: Abstract = 'NA'
                Link= 'https://www.ncbi.nlm.nih.gov/pubmed/{}'.format(PMID)   


                self.mainDict[PMID] = {'PMID': PMID, 'Title':Title ,
                                'JournalName':JournalName ,
                                  'Creation Date':dateC ,
                                'Publication Date':dateP , 
                                'Abstract':Abstract , 
                                'Link':Link,
                                'Tag':Tag   }
                self.NumNew +=1
                print('Added new PMID: {}'.format(PMID))
                self.Log.append('Added new PMID: {}'.format(PMID))


In [3]:
if __name__ == '__main__':
    Today =datetime.datetime.now()
    StartDate = Today - datetime.timedelta(days=3)

    Today = Today.strftime("%Y/%m/%d")
    StartDate = StartDate.strftime("%Y/%m/%d")
    
    COVID(StartDate , Today )

(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] )))) AND  AND "2020/03/25"[MHDA] : "2020/03/28"[MHDA]
Found 172 results
Going to download record 1 to 10
PMID [32081636] exists
PMID [32209231] exists
PMID [32194944] exists
PMID [32207910] exists
PMID [32174053] exists
PMID [32134278] exists
PMID [32081569] exists
PMID [32207679] exists
PMID [32191676] exists
PMID [32207377] exists
Going to download record 11 to 20
PMID [32191830] exists
PMID [32216248] exists
PMID [32119825] exists
PMID [32100486] exists
PMID [32215622] exists
PMID [32206694] exists
PMID [32151324] exists
PMID [32191813] exists
PMID [32105090] exists
PMID [32215956] exists
Going to download record 21 to 30
PMID [32204750] exists
PMID [32200398] exists
PMID [32209

PMID [32209539] exists
PMID [32209450] exists
PMID [32209449] exists
PMID [32208486] exists
PMID [32208485] exists
PMID [32207816] exists
PMID [32207810] exists
PMID [32207809] exists
PMID [32207808] exists
PMID [32207807] exists
Going to download record 121 to 130
PMID [32207806] exists
PMID [32207804] exists
PMID [32207802] exists
PMID [32207800] exists
PMID [32207799] exists
PMID [32207798] exists
PMID [32207797] exists
PMID [32207796] exists
PMID [32207795] exists
PMID [32207794] exists
Going to download record 131 to 140
PMID [32207793] exists
PMID [32207792] exists
PMID [32207791] exists
PMID [32207790] exists
PMID [32207789] exists
PMID [32207788] exists
PMID [32207787] exists
PMID [32207786] exists
PMID [32207785] exists
PMID [32207783] exists
Going to download record 141 to 150
PMID [32207782] exists
PMID [32207781] exists
PMID [32207779] exists
PMID [32207778] exists
PMID [32207777] exists
PMID [32207776] exists
PMID [32207775] exists
PMID [32207774] exists
PMID [32207769] ex

Found 0 results
(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] )))) AND case control study AND "2020/03/25"[MHDA] : "2020/03/28"[MHDA]
Found 5 results
Going to download record 1 to 5
PMID [32198267] exists
PMID [32213332] exists
PMID [32204756] exists
PMID [32087820] exists
PMID [32191764] exists
(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] )))) AND epidemiology AND "2020/03/25"[MHDA] : "2020/03/28"[MHDA]
Found 51 results
Going to download record 1 to 10
PMID [3208163

In [6]:
#COVID('2020/01/27' , '2020/03/28' )

(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] )))) AND  AND "2020/01/27"[MHDA] : "2020/03/28"[MHDA]
Found 1068 results
Going to download record 1 to 10
Added new PMID: 32081636
Added new PMID: 32160889
Added new PMID: 32106567
Added new PMID: 32173241
Added new PMID: 32209231
Added new PMID: 32100667
Added new PMID: 32166607
Added new PMID: 32062645
Added new PMID: 32194944
Added new PMID: 32169119
Going to download record 11 to 20
Added new PMID: 32070465
Added new PMID: 32207910
Added new PMID: 32147628
Added new PMID: 32172672
Added new PMID: 32178768
Added new PMID: 32190290
Added new PMID: 32172669
Added new PMID: 32143123
Added new PMID: 32156648
Added new PMID: 32092911
Going to download record 21 to 30
Added new PMID: 

Added new PMID: 32184131
Added new PMID: 32184128
Added new PMID: 32183941
Added new PMID: 32183937
Added new PMID: 32183935
Added new PMID: 32183934
Added new PMID: 32183930
Added new PMID: 32183920
Added new PMID: 32183901
Added new PMID: 32183864
Going to download record 291 to 300
Added new PMID: 32183357
Added new PMID: 32183172
Added new PMID: 32182811
Added new PMID: 32182131
Added new PMID: 32181990
Added new PMID: 32181969
Added new PMID: 32181904
Added new PMID: 32181903
Added new PMID: 32181901
Added new PMID: 32181874
Going to download record 301 to 310
Added new PMID: 32181873
Added new PMID: 32181864
Added new PMID: 32181795
Added new PMID: 32181672
Added new PMID: 32181577
Added new PMID: 32181488
Added new PMID: 32181483
PMID [32180426] exists
Added new PMID: 32180292
Added new PMID: 32180175
Going to download record 311 to 320
Added new PMID: 32180173
Added new PMID: 32180140
Added new PMID: 32179910
Added new PMID: 32179908
Added new PMID: 32179860
Added new PMID: 321

Added new PMID: 32141062
Added new PMID: 32141058
Added new PMID: 32140538
Added new PMID: 32139521
PMID [32139372] exists
Added new PMID: 32139299
Added new PMID: 32138488
Added new PMID: 32138266
Added new PMID: 32135587
Added new PMID: 32135586
Going to download record 581 to 590
Added new PMID: 32135585
Added new PMID: 32135584
Added new PMID: 32135077
Added new PMID: 32134909
Added new PMID: 32134861
Added new PMID: 32134800
Added new PMID: 32134681
Added new PMID: 32134381
PMID [32134278] exists
Added new PMID: 32134205
Going to download record 591 to 600
Added new PMID: 32134116
Added new PMID: 32134111
Added new PMID: 32133964
Added new PMID: 32133962
Added new PMID: 32133833
Added new PMID: 32133832
Added new PMID: 32133578
Added new PMID: 32133153
Added new PMID: 32133152
Added new PMID: 32132747
Going to download record 601 to 610
Added new PMID: 32132744
PMID [32132521] exists
Added new PMID: 32132379
Added new PMID: 32132196
Added new PMID: 32132184
Added new PMID: 3213191

Added new PMID: 32074444
Added new PMID: 32073631
Added new PMID: 32073161
Added new PMID: 32072794
Added new PMID: 32072569
Added new PMID: 32072255
Added new PMID: 32071427
Added new PMID: 32071063
Added new PMID: 32070753
Added new PMID: 32070466
Going to download record 871 to 880
PMID [32070465] exists
Added new PMID: 32070391
Added new PMID: 32068012
Added new PMID: 32067043
Added new PMID: 32066541
Added new PMID: 32066526
Added new PMID: 32066525
Added new PMID: 32065221
Added new PMID: 32065057
Added new PMID: 32065055
Going to download record 881 to 890
Added new PMID: 32064855
Added new PMID: 32064853
Added new PMID: 32064795
Added new PMID: 32062875
PMID [32062645] exists
Added new PMID: 32061335
Added new PMID: 32061313
Added new PMID: 32061311
Added new PMID: 32061284
Added new PMID: 32061201
Going to download record 891 to 900
Added new PMID: 32061200
Added new PMID: 32061198
Added new PMID: 32060933
Added new PMID: 32060789
Added new PMID: 32059801
Added new PMID: 32059

Added new PMID: 32213327
Added new PMID: 32213326
Added new PMID: 32213325
Added new PMID: 32213320
Added new PMID: 32211889
Added new PMID: 32211831
Added new PMID: 32211830
Added new PMID: 32211823
Added new PMID: 32211819
Added new PMID: 32210389
Going to download record 81 to 90
Added new PMID: 32210388
Added new PMID: 32210387
Added new PMID: 32210386
Added new PMID: 32210385
Added new PMID: 32210384
Added new PMID: 32210383
Added new PMID: 32210382
Added new PMID: 32210381
Added new PMID: 32210379
Added new PMID: 32210378
Going to download record 91 to 100
Added new PMID: 32210377
Added new PMID: 32210376
Added new PMID: 32210375
Added new PMID: 32210374
Added new PMID: 32210373
Added new PMID: 32210372
Added new PMID: 32210371
Added new PMID: 32209588
Added new PMID: 32209559
Added new PMID: 32209558
Going to download record 101 to 110
Added new PMID: 32209552
Added new PMID: 32209551
Added new PMID: 32209549
Added new PMID: 32209548
Added new PMID: 32209547
Added new PMID: 3220

...

Added new PMID: 31706930
Added new PMID: 31704264
Added new PMID: 31704263
Added new PMID: 31704113
Added new PMID: 31680014
Added new PMID: 31680013
Added new PMID: 31676323
Added new PMID: 31672337
Added new PMID: 31666724
Added new PMID: 31666721
Going to download record 2691 to 2700
Added new PMID: 31664261
Added new PMID: 31664194
Added new PMID: 31653557
Added new PMID: 31653556
Added new PMID: 31645766
Added new PMID: 31645763
Added new PMID: 31645760
Added new PMID: 31645758
Added new PMID: 31645757
Added new PMID: 31634902
Going to download record 2701 to 2710
Added new PMID: 31630984
Added new PMID: 31630983
Added new PMID: 31630982
Added new PMID: 31619797
Added new PMID: 31619796
Added new PMID: 31619795
Added new PMID: 31619791
Added new PMID: 31619789
Added new PMID: 31619788
Added new PMID: 31619787
Going to download record 2711 to 2720
Added new PMID: 31618757
Added new PMID: 31616104
Added new PMID: 31616103
Added new PMID: 31616102
Added new PMID: 31616098
Added new P

Added new PMID: 31068699
Added new PMID: 31068698
Added new PMID: 31068697
Added new PMID: 31068696
Added new PMID: 31068695
Added new PMID: 31061499
Added new PMID: 31060978
Added new PMID: 31043749
Added new PMID: 31043747
Added new PMID: 31043746
Going to download record 2981 to 2990
Added new PMID: 31043742
Added new PMID: 31043741
Added new PMID: 31043740
Added new PMID: 31022719
Added new PMID: 31019318
Added new PMID: 31019307
Added new PMID: 31019297
Added new PMID: 31015697
Added new PMID: 31015695
Added new PMID: 31015692
Going to download record 2991 to 3000
Added new PMID: 31015690
Added new PMID: 31003964
Added new PMID: 30996349
Added new PMID: 30995674
Added new PMID: 30959515
Added new PMID: 30794309
Added new PMID: 30765327
Added new PMID: 32145833
Added new PMID: 32145832
Added new PMID: 29973369
Going to download record 3001 to 3006
Added new PMID: 29496303
Added new PMID: 29153410
Added new PMID: 29042356
Added new PMID: 28855079
Added new PMID: 28284589
Added new P

PMID [32152595] exists
PMID [32151613] exists
PMID [32151326] exists
PMID [32151325] exists
PMID [32151324] exists
PMID [32150618] exists
PMID [32149043] exists
PMID [32149037] exists
PMID [32148173] exists
PMID [32148172] exists
Going to download record 131 to 140
PMID [32146721] exists
PMID [32146694] exists
PMID [32146445] exists
PMID [32145772] exists
PMID [32145718] exists
PMID [32145466] exists
PMID [32145187] exists
PMID [32145186] exists
PMID [32143990] exists
PMID [32143519] exists
Going to download record 141 to 150
PMID [32142972] exists
PMID [32142596] exists
PMID [32141588] exists
PMID [32141570] exists
PMID [32141569] exists
PMID [32141062] exists
PMID [32139521] exists
PMID [32139372] exists
PMID [32138488] exists
PMID [32138266] exists
Going to download record 151 to 160
PMID [32135587] exists
PMID [32135586] exists
PMID [32135077] exists
PMID [32134909] exists
PMID [32134861] exists
PMID [32134111] exists
PMID [32133833] exists
PMID [32133832] exists
PMID [32132747] ex

Found 8 results
Going to download record 1 to 8
PMID [32180426] exists
PMID [32050635] exists
PMID [32151335] exists
PMID [32196655] exists
PMID [32119083] exists
PMID [32151334] exists
PMID [32114744] exists
PMID [32181904] exists
(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] )))) AND (treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields]) AND "2020/01/27"[MHDA] : "2020/03/28"[MHDA]
Found 529 results
Going to download record 1 to 10
PMID [32081636] exists
PMID [32160889] exists
PMID [32173241] exists
PMID [32106567] exists
PMID [32194944] exists
PMID [32166607] exists
PMID [32134278] exists
PMID [32169119] exists
PMID [32207910] exists
PMID [32147628] exists
Going to download record 11 

PMID [32133832] exists
PMID [32133152] exists
PMID [32132747] exists
PMID [32132521] exists
PMID [32131908] exists
PMID [32130833] exists
PMID [32130038] exists
PMID [32129843] exists
PMID [32129583] exists
PMID [32129518] exists
Going to download record 301 to 310
PMID [32127713] exists
PMID [32127711] exists
PMID [32127666] exists
PMID [32127517] exists
PMID [32127123] exists
PMID [32125873] exists
PMID [32125362] exists
PMID [32125140] exists
PMID [32125132] exists
PMID [32125131] exists
Going to download record 311 to 320
PMID [32125130] exists
PMID [32125126] exists
PMID [32124990] exists
PMID [32124506] exists
PMID [32124179] exists
PMID [32124000] exists
PMID [32122812] exists
PMID [32122113] exists
PMID [32121356] exists
PMID [32120458] exists
Going to download record 321 to 330
PMID [32119961] exists
PMID [32119825] exists
PMID [32119824] exists
PMID [32118644] exists
PMID [32118640] exists
PMID [32118639] exists
PMID [32118394] exists
PMID [32118391] exists
PMID [32118390] ex

In [14]:
def MakeTemplate():
    headerslist = []
    for H in ('PMID',
              'Title',
              'Link',
                'JournalName',
              'Creation Date',
                'Publication Date',
                'Abstract',
                'Tag'):
        print(H)
        headers = '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/{}", "noInherit,noTruncate")'.format(H)
        headerslist.append(headers)

    headerslist
    with open('template.csv' ,'w'  , newline='' ) as fp:
        W = csv.writer(fp, delimiter=';')
        W.writerow(headerslist)

# google

In [6]:
import requests
from bs4 import BeautifulSoup

query = '"covid-19"'
url =  'https://scholar.google.com/scholar?start=0&q='+ query + '&hl=en&scisbd=1&as_sdt=1,5&as_vis=1&ie=UTF-8&oe=UTF-8&hl=en&btnG=Search'

content = requests.get(url).text
page = BeautifulSoup(content, 'html')
results = []
for entry in page.find_all("h3", attrs={"class": "gs_rt"}):
    results.append({"title": entry.a.text, "url": entry.a['href']})

In [7]:
for entry in page.find_all(attrs={"class": "gs_rs"}):
    maintext = entry.get_text()
    D = maintext.split(' ')[0]
    print(D)
    print(entry.get_text())
    print()

2
2 days ago - The coronavirus disease (COVID-19) worldwide outbreak has led to a dramatic challenge for all healthcare systems, including inflammatory bowel disease (IBD) centres. Here, we describe the fast changes and clinical issues that IBD specialists could …

3
3 days ago - This study aims to analyze the different clinical characteristics between children and their families infected with severe acute respiratory syndrome coronavirus 2. Clinical data from nine children and their 14 families were collected, including general …

3
3 days ago -  … unict.it (DG), arescifina@unict.it (AR) Abstract: The current emergency due
to the worldwide spread of the COVID-19 caused by the new SARS-CoV-2 is a great
concern for global public health. Already in the past, the outbreak … 


3
3 days ago - Since the outbreak of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) in December 2019, coronavirus disease 2019 (COVID-19) has ravaged the world and caused a global pandemic. Our healthc

In [8]:
results

[{'title': 'Management of IBD during the COVID-19 outbreak: resetting clinical priorities',
  'url': 'https://www.nature.com/articles/s41575-020-0294-8'},
 {'title': 'The different clinical characteristics of corona virus disease cases between children and their families in China–the character of children with COVID-19',
  'url': 'https://www.tandfonline.com/doi/abs/10.1080/22221751.2020.1744483'},
 {'title': 'Inhibitors of SARS-CoV-2 Main Protease from a Library of Marine Natural Products: A Virtual Screening and Molecular Modeling Study',
  'url': 'https://www.preprints.org/manuscript/202003.0372'},
 {'title': 'COVID-19: An Unintended Force for Medical Revolution?',
  'url': 'https://www.invasivecardiology.com/index.php/commentary/covid-19-unintended-force-medical-revolution'},
 {'title': 'Renin-Angiotensin-Aldosterone System: Double-Edged Sword in COVID-19 Infection',
  'url': 'https://www.preprints.org/manuscript/202003.0365'},
 {'title': '…\xa0reducing the total infection cases ou